In [4]:
import argparse
from collections import defaultdict
from pathlib import Path
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from joblib import Parallel, delayed
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import Isomap
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from mcda.dataset import MCDADataset
from mcda.report import calculate_heuristics
from mcda.uta import Criterion, check_uta_feasibility
from methods.autoencoder import DominanceAutoEncoder
from methods.mvu import MaximumVarianceUnfolding

In [8]:
dataset = MCDADataset.read_csv("./data/regions-italy/dataset.csv")
df = dataset.data

In [9]:
def is_dominating(a_values: List[float], b_values: List[float]) -> bool:
    return all(a >= b for a, b in zip(a_values, b_values)) and any(a > b for a, b in zip(a_values, b_values))


domination_org = []
for alt_1 in df.index:
    for alt_2 in df.index:
        if alt_1 == alt_2:
            continue

        if is_dominating(df.loc[alt_1, :].tolist(), df.loc[alt_2, :].tolist()):
            print(alt_1, " > ", alt_2)
            domination_org.append((alt_1, alt_2))

print(f"Oryginalnie wszystkich dominacji: {len(domination_org)}")

N = 2
methods = {
    "PCA": Pipeline([("scaler", StandardScaler()), ("pca", PCA(n_components=2, random_state=42))]),
    "KernelPCA": Pipeline(
        [("scaler", StandardScaler()), ("kpca", KernelPCA(n_components=2, random_state=42, kernel="sigmoid"))]
    ),
    "Isomap": Pipeline([("scaler", StandardScaler()), ("isomap", Isomap(n_components=2))]),
    "MVU": Pipeline([("scaler", StandardScaler()), ("mvu", MaximumVarianceUnfolding(n_components=2, seed=42))]),
    "DAE": Pipeline(
        [
            ("scaler", StandardScaler()),
            ("dae", DominanceAutoEncoder(latent_dim=2, num_epochs=1000, random_state=42, verbose=False)),
        ]
    ),
}
for method_name, method in methods.items():
    df_m = pd.DataFrame(method.fit_transform(df), index=df.index, columns=[f"Dim {i + 1}" for i in range(N)])
    counter = 0
    for alt_1, alt_2 in domination_org:
        if not is_dominating(df_m.loc[alt_1, :].tolist(), df_m.loc[alt_2, :].tolist()):
            # print("!!!", alt_1, " > ", alt_2)
            counter += 1
    print(f"{method_name.ljust(12, ' ')}niespełnionych dominacji: {counter}")

a1  >  a10
a2  >  a6
a2  >  a7
a2  >  a11
a3  >  a5
a3  >  a6
a3  >  a7
a3  >  a10
a3  >  a11
a4  >  a5
a4  >  a6
a4  >  a7
a4  >  a11
a5  >  a11
a8  >  a11
a15  >  a18
a17  >  a18
Oryginalnie wszystkich dominacji: 17
PCA         niespełnionych dominacji: 0
KernelPCA   niespełnionych dominacji: 9
Isomap      niespełnionych dominacji: 14
MVU         niespełnionych dominacji: 10
DAE         niespełnionych dominacji: 0
